In [ ]:
import pandas as pd
import io
import numpy as np
from google.colab import files
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Assuming the file is named 'Customer_Data.csv'
uploaded = files.upload()
df= pd.read_csv(next(iter(uploaded.keys())))
# Display the first few rows to ensure it's loaded correctly
print(df.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving english_movies.csv to english_movies (2).csv
                              title  \
0                         The Flash   
1                            Barbie   
2  Transformers: Rise of the Beasts   
3                The Little Mermaid   
4      Ruby Gillman, Teenage Kraken   

                                            overview release_date  \
0  When his attempt to save his family inadverten...   2023-06-13   
1  Barbie and Ken are having the time of their li...   2023-07-19   
2  When a new threat capable of destroying the en...   2023-06-06   
3  The youngest of King Triton’s daughters, and t...   2023-05-18   
4  Ruby Gillman, a sweet and awkward high school ...   2023-06-28   

                                genres  popularity  vote_average  vote_count  
0   Action, Adventure, Science Fiction    4631.142           6.9        1773  
1           Comedy, Adventure, Fantasy    4493.487           7.6        1621  
2   Action, Adventure, Science Fiction    4090.661           

In [ ]:
# Descriptive statistics for numerical features
print(df.describe())

# To include all columns including categorical data, use include='all'
print(df.describe(include='all'))


         popularity  vote_average    vote_count
count  10000.000000  10000.000000  10000.000000
mean      29.335884      6.282290   1548.338600
std      105.733120      1.228712   2884.216216
min        6.479000      0.000000      0.000000
25%       12.478750      5.800000    174.000000
50%       16.578000      6.400000    509.000000
75%       25.806500      7.000000   1521.000000
max     4631.142000     10.000000  34102.000000
                    title                                           overview  \
count               10000                                               9995   
unique               9627                                               9993   
top     A Christmas Carol  This documentary focuses on the actors and the...   
freq                    4                                                  2   
mean                  NaN                                                NaN   
std                   NaN                                                NaN   
min     

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Check for null values in the dataset
print(df.isnull().sum())


title            0
overview         5
release_date    18
genres          22
popularity       0
vote_average     0
vote_count       0
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Remove rows where any of the specified columns have a null value
df.dropna(subset=['overview', 'release_date', 'genres'], inplace=True)

# Alternatively, to remove rows where any cell across the DataFrame has a null value
# df.dropna(inplace=True)

# Check the dimensions of the DataFrame after dropping null values
print(df.shape)

# Re-check for null values to ensure rows have been removed correctly
print(df.isnull().sum())


(9959, 7)
title           0
overview        0
release_date    0
genres          0
popularity      0
vote_average    0
vote_count      0
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Assuming 'genres' is the column containing items for association analysis
# and it contains data like "['Action', 'Adventure', 'Sci-Fi']"

# 1. Extract the 'genres' column and convert it to a list of lists
transactions = df['genres'].str.strip('[]').str.replace("'", "").str.split(', ').tolist()

# 2. One-hot encode the transactions using TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# 3. Create a DataFrame from the encoded data
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# 4. Apply apriori and association_rules to the encoded DataFrame
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# 5. Display rules sorted by lift
rules.sort_values('lift', ascending=False, inplace=True)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                        antecedents       consequents   support  confidence  \
39      (Adventure, Comedy, Family)       (Animation)  0.022894    0.649573   
47     (Fantasy, Animation, Comedy)          (Family)  0.012551    0.912409   
38   (Adventure, Animation, Comedy)          (Family)  0.022894    0.912000   
25              (Comedy, Animation)          (Family)  0.045386    0.886275   
41           (Adventure, Animation)  (Comedy, Family)  0.022894    0.517007   
42  (Adventure, Fantasy, Animation)          (Family)  0.012652    0.845638   
19              (Adventure, Family)       (Animation)  0.037253    0.578783   
18           (Adventure, Animation)          (Family)  0.037253    0.841270   
28             (Fantasy, Animation)          (Family)  0.022894    0.805654   
2                          (Family)       (Animation)  0.080129    0.540284   
1                       (Animation)          (Family)  0.080129    0.787759   
26                 (Comedy, Family)       (Animation

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Filling NaN values with an empty string if any
df['overview'] = df['overview'].fillna('')

# Creating the TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Function to get movie recommendations based on cosine similarity scores
def get_recommendations(title):
    # Get the index of the movie that matches the title
    idx = df.index[df['title'] == title].tolist()[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Creating the TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def get_recommendations(title):
    # Get the index of the movie that matches the title
    idx = df.index[df['title'] == title].tolist()[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Replace 'Movie Title Here' with a real title from your dataset
recommended_movies = get_recommendations('After')
print(recommended_movies)

58                       After We Fell
56                   After We Collided
64                    After Everything
44                    After Ever Happy
6167                       Re-Animator
3637    Cranston Academy: Monster Zone
6368             The Constant Gardener
8530            A Little Bit of Heaven
9467                           Spartan
1783                   I'm Not Ashamed
Name: title, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
